In [1]:
!apt install -y libboost-all-dev
!pip install -U pip
!pip install "pybind11[global]"
!pip install aijack

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libboost-all-dev is already the newest version (1.74.0.3ubuntu7).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 5.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.7/425.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.6/142.6 kB 1.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 14.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Prepa

In [2]:
from sklearn.metrics import roc_auc_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from aijack.attack.membership import ShadowMembershipInferenceAttack
from aijack.utils.utils import TorchClassifier, NumpyDataset

In [4]:
X_train = pd.read_csv('/content/real.csv').astype('float')
X_test = pd.read_csv('/content/real_test.csv').astype('float')
X_shadow = pd.read_csv('/content/syn.csv').astype('float')

y_train = np.array([0 if x == 0 else 1 for x in X_train['BSPT_DEAD_YMD']]).astype('int')
y_test = np.array([0 if x == 0 else 1 for x in X_test['BSPT_DEAD_YMD']]).astype('int')
y_shadow = np.array([0 if x == 0 else 1 for x in X_shadow['BSPT_DEAD_YMD']]).astype('int')

In [5]:
X_train = np.array(X_train)
X_test = np.array(X_test)
X_shadow = np.array(X_shadow)

In [6]:
# Train the victim

clf = SVC(probability=True)
clf.fit(X_train, y_train)
clf.score(X_train, y_train), clf.score(X_test, y_test)

(0.8945868945868946, 0.9088319088319088)

In [7]:
# Train the attacker

shadow_models = [SVC(probability=True) for _ in range(2)]
attack_models = [SVC(probability=True) for _ in range(10)]

attacker = ShadowMembershipInferenceAttack(clf, shadow_models, attack_models)
attacker.fit(X_shadow, y_shadow)

In [8]:
# Get the attack result of membership inference
in_result = attacker.predict(clf.predict_proba(X_train), y_train)
out_result = attacker.predict(clf.predict_proba(X_test), y_test)

in_label = np.ones(in_result.shape[0])
out_label = np.zeros(out_result.shape[0])

accuracy_score(
    np.concatenate([in_label, out_label]), np.concatenate([in_result, out_result])
)

0.49952516619183285